## RAG
`RAG - это техника, повышающая производительность языковых моделей путём предоставления модели контекста вместе с вопросом.`

Последовательность действий:
1. Передадим модели информацию о нашем заводе без дополнительного тюнинга;
2. Создадим базу векторов, где будут храниться ембеддинги ранее заданных вопросов (кэш);
3. При обращении к модели, будем проверять, задавались ли ранее похожие вопросы. Если да, то отдаём ранее сгенерированные ответы.

Зачем использовать кэш?
* Чтобы увеличить скорость ответов для вопросов, которые задавались ранее.
* Снизить затраты при использовании платных API (GTP-3.5, GPT-4) для ответов на однотипные и повторяющиеся вопросы.

В качестве модели мы будем использовать адаптивные веса русскоязычной "saiga_mistral_7b_lora" (https://huggingface.co/IlyaGusev/saiga_mistral_7b_lora) (Лицензия CC BY 4.0), которые натренировал Илья Гусев @Takagi

Оригинальная модель - "Mistral-7B-OpenOrca" (https://huggingface.co/Open-Orca/Mistral-7B-OpenOrca)

### Импорты

In [3]:
import os

import torch
import numpy as np

from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F
from langchain.prompts import PromptTemplate

# !pip install faiss-cpu
import faiss

In [30]:
# utils
cache_dir = r'E:\dev\0_MODELS_HF'
os.environ["TRANSFORMERS_CACHE"] = cache_dir
device_map = {"": 0}

# db
answers_file_path = './database/answers.txt'
emb_file_path = "./database/emb_database.npy"

# models
base_model_name = "Open-Orca/Mistral-7B-OpenOrca"
adapt_model_name = "IlyaGusev/saiga_mistral_7b_lora"

sentence_model_name = 'sentence-transformers/all-MiniLM-L6-v2'

### Загружаем модель для генерации ответа

In [5]:
# load models and tokenizer

tokenizer = AutoTokenizer.from_pretrained(
    base_model_name,
    trust_remote_code=True,
    cache_dir=cache_dir
)
tokenizer.pad_token = tokenizer.eos_token

model = AutoPeftModelForCausalLM.from_pretrained(
    adapt_model_name,
    device_map=device_map,
    torch_dtype=torch.bfloat16,
    cache_dir=cache_dir
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Загружаем модель для получения эмбеддингов

In [8]:
sent_tokenizer = AutoTokenizer.from_pretrained(sentence_model_name, cache_dir=cache_dir)
sent_model = AutoModel.from_pretrained(sentence_model_name, cache_dir=cache_dir)

In [9]:
# функция для получения эмбеддингов.
# На вход подаём строку, на выходе получаем torch.tensor размерностью (1, 384)

def get_embedding(sentence):
    
    # Mean Pooling - Take attention mask into account for correct averaging
    def _mean_pooling(model_output, attention_mask):
        token_embeddings = model_output[0] # First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    # Tokenize sentences
    encoded_input = sent_tokenizer([sentence], padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = sent_model(**encoded_input)

    # Perform pooling
    sentence_embeddings = _mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

    return sentence_embeddings

### Создаём базу для эмбеддингов.

In [61]:
# Функции для сохранения и загрузки db
def update_db(answers, emb_database):
    with open(answers_file_path, 'w') as file:
        file.writelines(answer + '\n' for answer in answers)
    np.save(emb_file_path, emb_database.numpy())

def load_db(answers_file_path, emb_file_path):
    if os.path.exists(answers_file_path) and os.path.exists(emb_file_path):
        with open(answers_file_path, 'r') as file:
            answers = [line.strip() for line in file.readlines()]
        emb_database = torch.tensor(np.load(emb_file_path), dtype=torch.float32)
    else:
        answers = []
        emb_database = torch.empty((0, 384), dtype=torch.float32)
    return answers, emb_database    

### Prompting

Подготовим краткое описание вымышленной компании, которое будем подавать модели вместе с вопросами, чтобы модель была в курсе контекста.

```
user: Компания "Стальной Щит" специализируется на разработке и производстве передовой военной техники. Наше предприятие расположено на улице Технологическая, 123, в городе Защитоград. До компании Вы можете добраться до нас следующими способами: 1. На метро: станция "Вымышленная", 10 выход, прямо и налево. 2. На автобусе: автобус №100 "Атаковоево - Защитоград". С конечной остановки прямо и направо. Мы занимаемся созданием инновационных боевых машин, включая танки, боевые вертолеты, беспилотные летательные аппараты и системы киберзащиты. Компания "Стальной Щит" стремится к высочайшему качеству продукции, применяя передовые технологии и сотрудничая с лучшими специалистами в области военной промышленности. Мы гордимся нашими инновационными решениями, обеспечивающими безопасность и надежность наших вооруженных сил. Вы можете связаться с нами по телефону +1234567890 или по электронной почте info@steelshield.com, чтобы узнать больше о нашей продукции, услугах и возможностях сотрудничества. Наши специалисты готовы ответить на ваши вопросы о нашей технике, ее характеристиках, ценах и условиях поставки. {question}\nbot: Вот ответ на ваш вопрос длиной не более 10 слов:")
```

Обернём промт в PromptTemplate из библиотеки langchain. Это аналог f-строки, только с возможностью передавать строку, как зависимую переменную, с последующей передачей ей аргумента.

In [ ]:
info_prompt_less10 = PromptTemplate.from_template(r'user: Компания "Стальной Щит" специализируется на разработке и производстве передовой военной техники. Наше предприятие расположено на улице Технологическая, 123, в городе Защитоград. До компании Вы можете добраться до нас следующими способами: 1. На метро: станция "Вымышленная", 10 выход, прямо и налево. 2. На автобусе: автобус №100 "Атаковоево - Защитоград". С конечной остановки прямо и направо. Мы занимаемся созданием инновационных боевых машин, включая танки, боевые вертолеты, беспилотные летательные аппараты и системы киберзащиты. Компания "Стальной Щит" стремится к высочайшему качеству продукции, применяя передовые технологии и сотрудничая с лучшими специалистами в области военной промышленности. Мы гордимся нашими инновационными решениями, обеспечивающими безопасность и надежность наших вооруженных сил. Вы можете связаться с нами по телефону +1234567890 или по электронной почте info@steelshield.com, чтобы узнать больше о нашей продукции, услугах и возможностях сотрудничества. Наши специалисты готовы ответить на ваши вопросы о нашей технике, ее характеристиках, ценах и условиях поставки. {question}\nbot: Вот ответ на ваш вопрос длиной не более 10 слов:")')


# Функция для генерации ответа моделью и парсинг ответа
def get_answer(info_prompt, question):
    
    prompt = info_prompt.format(question=question)   
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), 
                            top_p=0.5,
                            temperature=0.3,
                            attention_mask=inputs["attention_mask"],
                            max_new_tokens=50,
                            pad_token_id=tokenizer.eos_token_id,
                            do_sample=True)

    output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    parsed_answer = output.split("Вот ответ на ваш вопрос длиной не более 10 слов:")[1].strip()

    if "bot:" in parsed_answer:
        parsed_answer = parsed_answer.split("bot:")[0].strip()

    return parsed_answer

### Пайплайн:

In [66]:
def get_cos_sim(emb, emb_database):
    return F.cosine_similarity(emb_database, emb, dim=1, eps=1e-8)

def pipe(question, answers):
    global emb_database
    emb = get_embedding(question)                                       # Создаем эмбеддинг вопроса
    cos_sim = get_cos_sim(emb, emb_database)
    if cos_sim.numel() == 0:
        cos_sim = torch.tensor([0.0])
    max_value, max_index = torch.max(cos_sim, dim=0)                    # Получаем самый похожий вопрос и индекс сохраненного ответа

    if max_value > 0.83:
        answer = answers[max_index]
        print(f'cos_sim={max_value}\nОтвет из DB: {answer}')            # Если есть похожий вопрос то выдаем закэшированный ответ из БД
    else:
        answer = get_answer(info_prompt_less10, question)               # Если нет - выдаем сгенерированный ответ нашей модели
        emb_database = torch.cat((emb_database, emb), dim=0)            # Сохраняем эмбеддинг в БД
        answers.append(answer)                                          # Сохраняем ответ от модели
        update_db(answers, emb_database) # Обновляем БД
        print(f'cos_sim={max_value}\nОтвет из MODEL: {answer}')
    print()

In [85]:
answers, emb_database = load_db(answers_file_path, emb_file_path)

In [87]:
question = "Название компании"
pipe(question, answers)

cos_sim=0.8721624612808228
Ответ из DB: "Стальной Щит".



# Завернем все в класс и попробуем запустить
(Файл retrieval_augmented_generation.py)

### TODO: Попробовать запустить в oobaboga

In [1]:
from pathlib import Path
import os
import torch
import numpy as np
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F
from langchain.prompts import PromptTemplate
from tqdm.auto import tqdm
# import faiss

cur_dir = Path.cwd()

# utils
cache_dir = 'E:/dev/0_MODELS_HF' if os.path.exists('E:/dev/0_MODELS_HF') else '~/.cache/huggingface/transformers'
print('cache_dir:', cache_dir)

os.environ["TRANSFORMERS_CACHE"] = cache_dir
device_map = {"": 0}

# db
answers_file_path = cur_dir.joinpath('database', 'answers.txt')
emb_file_path = cur_dir.joinpath('database', 'emb_database.npy')

# models
base_model_name = "Open-Orca/Mistral-7B-OpenOrca"
adapt_model_name = "IlyaGusev/saiga_mistral_7b_lora"

sentence_model_name = 'sentence-transformers/all-MiniLM-L6-v2'


class RAG_FOR_COMPANY:
    def __init__(self) -> None:

        self.tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True, cache_dir=cache_dir)
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.model = AutoPeftModelForCausalLM.from_pretrained(adapt_model_name, device_map=device_map, torch_dtype=torch.bfloat16, cache_dir=cache_dir)
        self.sent_tokenizer = AutoTokenizer.from_pretrained(sentence_model_name, cache_dir=cache_dir)
        self.sent_model = AutoModel.from_pretrained(sentence_model_name, cache_dir=cache_dir)

        self.answers, self.emb_database = self.load_db(answers_file_path, emb_file_path)

        self.info_prompt_less10 = PromptTemplate.from_template('user: Компания "Стальной Щит" специализируется на разработке и производстве передовой военной техники. Наше предприятие расположено на улице Технологическая, 123, в городе Защитоград. Вы можете добраться до нас следующими способами: На метро: станция "Вымышленная" 10 выход, на автобусе: автобус №100 "Атаковоево - Защитоград". Мы занимаемся созданием инновационных боевых машин, включая танки, боевые вертолеты, беспилотные летательные аппараты и системы киберзащиты. Вы можете связаться с нами по телефону: +79876543210 или по почте: zashitograd@sb.ru. {question}\nbot: Вот ответ на ваш вопрос длиной не более 10 слов:"')

    def get_embedding(self, sentence):
        def _mean_pooling(model_output, attention_mask):
            token_embeddings = model_output[0] # First element of model_output contains all token embeddings
            input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
            return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

        encoded_input = self.sent_tokenizer([sentence], padding=True, truncation=True, return_tensors='pt')     # Tokenize sentences
        with torch.no_grad():                                                                                   # Compute token embeddings
            model_output = self.sent_model(**encoded_input)
        sentence_embeddings = _mean_pooling(model_output, encoded_input['attention_mask'])                      # Perform pooling
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)                                      # Normalize embeddings
        return sentence_embeddings

    def get_answer(self, question):
        prompt = self.info_prompt_less10.format(question=question)   
        inputs = self.tokenizer(prompt, return_tensors="pt").to("cuda")
        outputs = self.model.generate(input_ids=inputs["input_ids"].to("cuda"), 
                                top_p=0.5,
                                temperature=0.3,
                                attention_mask=inputs["attention_mask"],
                                max_new_tokens=50,
                                pad_token_id=self.tokenizer.eos_token_id,
                                do_sample=True)
        output = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        parsed_answer = output.split("Вот ответ на ваш вопрос длиной не более 10 слов:")[1].strip()
        if "bot:" in parsed_answer:
            parsed_answer = parsed_answer.split("bot:")[0].strip()
        return parsed_answer

    @staticmethod
    def update_db(answers, emb_database):
        with open(answers_file_path, 'w') as file:
            file.writelines(answer + '\n' for answer in answers)
        np.save(emb_file_path, emb_database.numpy())

    @staticmethod
    def load_db(answers_file_path, emb_file_path):
        if os.path.exists(answers_file_path) and os.path.exists(emb_file_path):
            with open(answers_file_path, 'r') as file:
                answers = [line.strip() for line in file.readlines()]
            emb_database = torch.tensor(np.load(emb_file_path), dtype=torch.float32)
        else:
            answers = []
            emb_database = torch.empty((0, 384), dtype=torch.float32)
        return answers, emb_database
    
    def get_cos_sim(self, emb):
        return F.cosine_similarity(self.emb_database, emb, dim=1, eps=1e-8)

    def pipe(self, question):
        emb = self.get_embedding(question)                                       # Создаем эмбеддинг вопроса
        cos_sim = self.get_cos_sim(emb)
        if cos_sim.numel() == 0:
            cos_sim = torch.tensor([0.0])
        max_value, max_index = torch.max(cos_sim, dim=0)                         # Получаем самый похожий вопрос и индекс сохраненного ответа

        if max_value > 0.83:
            source = 'DB'
            answer = self.answers[max_index]                                     # Если есть похожий вопрос то выдаем закэшированный ответ из БД
            return max_value, answer, source
        else:
            source = 'MODEL'
            answer = self.get_answer(question)                                   # Если нет - выдаем сгенерированный ответ нашей модели
            self.emb_database = torch.cat((self.emb_database, emb), dim=0)       # Сохраняем эмбеддинг в БД
            self.answers.append(answer)                                          # Сохраняем ответ от модели
            self.update_db(self.answers, self.emb_database)                      # Обновляем БД
            return max_value, answer, source

cache_dir: E:/dev/0_MODELS_HF


In [2]:
cls = RAG_FOR_COMPANY()

def get_response(question):
    print('Question:', question)
    max_value, answer, source = cls.pipe(question)
    response = f'Answer: {answer}\nCos_sim: {max_value}\nSource: {source}\n'
    print(response)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# Заполним базу данных самыми частыми вопросами
questions = [
"Какой адрес вашей компании?",
"Где находится ваша компания?",
"Какое местоположение вашей компании?",
"Где точно находится ваша компания?",
"Как добраться до вашей компании?",
"Как мне добраться до вашей компании?",
"На каком автобусе добраться до вашей компании?",
"Какие автобусы едут до вашей компанииа?",
"Что производят в вашей компании?",
"Какая продукция производится в вашей компании?",
"Какие товары производятся в вашей компании?",
"Что именно производится в вашей компании?",
"Какие изделия производятся в вашей компании?",
"Какую продукцию я могу найти в вашей компании?",
"Как можно с вами связаться?",
"Напишите пожалуйста ваши контакты",
"Дайте мне ваши контакты",
"Дайте мне номер телефона",
"Какая у вас почта?",
"Какой номер телефона?",
"Как с вами связаться?",
"Напишите почту для связи",
"Напиши почту и телефон"
]

In [4]:
for question in tqdm(questions):
    get_response(question)
    print()

  0%|          | 0/23 [00:00<?, ?it/s]

Question: Какой адрес вашей компании?
Answer: "улица Технологическая, 123, город Защитоград"
Cos_sim: 0.0
Source: MODEL


Question: Где находится ваша компания?
Answer: "Улица Технологическая, 123, Защитоград".
Cos_sim: 0.7932148575782776
Source: MODEL


Question: Какое местоположение вашей компании?
Answer: "улица Технологическая, 123, город Защитоград"
Cos_sim: 0.8784106373786926
Source: DB


Question: Где точно находится ваша компания?
Answer: "Улица Технологическая, 123, Защитоград".
Cos_sim: 0.978456974029541
Source: DB


Question: Как добраться до вашей компании?
Answer: "Можно добраться на автобусе №100".
Cos_sim: 0.8171380162239075
Source: MODEL


Question: Как мне добраться до вашей компании?
Answer: "Можно добраться на автобусе №100".
Cos_sim: 0.9820703268051147
Source: DB


Question: На каком автобусе добраться до вашей компании?
Answer: "Можно добраться на автобусе №100".
Cos_sim: 0.9330138564109802
Source: DB


Question: Какие автобусы едут до вашей компанииа?
Answer: "Мож

### Inference

In [5]:
question = 'Как я могу с вами связаться?'
get_response(question)

Question: Как я могу с вами связаться?
Answer: "Мы можем связаться по телефону или по почте".
Cos_sim: 0.9212259650230408
Source: DB



In [6]:
question = 'Напишите почту для связи'
get_response(question)

Question: Напишите почту для связи
Answer: "Спасибо, мы свяжемся с вами в ближайшее время"
Cos_sim: 0.9999999403953552
Source: DB



:) Ответы можно/нужно править в файле answers.txt